# Fine-tuning

### Imports

In [ ]:
from os.path import join
from json import load

from unsloth import FastVisionModel
from unsloth import is_bf16_supported
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

import torch

from scripts.authentication import authenticate_huggingface
from scripts.data import LesionData, DatasetAnalysis
from scripts.messages import create_training_message

import scripts.definitions as defs

### Autenticação

In [ ]:
authenticate_huggingface()

### Configuração

In [ ]:
QUANTIZATION = True



### Carregamento do dataset

In [ ]:
with open(join(defs.DATA_PATH, 'stt_data', 'training_dataset.json'), 'r', encoding='utf-8') as file:
    training_dataset = [LesionData(**data) for data in load(file)]

with open(join(defs.DATA_PATH, 'training_dataset_analysis.json'), 'r', encoding='utf-8') as file:
    training_dataset_analysis = DatasetAnalysis(**load(file))

with open(join(defs.DATA_PATH, 'stt_data', 'validation_dataset.json'), 'r', encoding='utf-8') as file:
    validation_dataset = [LesionData(**data) for data in load(file)]

with open(join(defs.DATA_PATH, 'validation_dataset_analysis.json'), 'r', encoding='utf-8') as file:
    validation_dataset_analysis = DatasetAnalysis(**load(file))

### Preparação das mensagens

In [ ]:

training_messages = []

for lesion_data in training_dataset:
    lesion_data.analysis = training_dataset_analysis
    training_messages.append(create_training_message(lesion_data))

validation_messages = generate_training_messages(validation_dataset, validation_data_size)

### Inicialização do LLaMa 3.2

In [ ]:
model, tokenizer = FastVisionModel.from_pretrained(
    'unsloth/Llama-3.2-11B-Vision-Instruct',
    load_in_4bit = False,
    use_gradient_checkpointing = 'unsloth',
)

### Configuração de treinamento

In [ ]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers=True,
    finetune_language_layers=True,
    finetune_attention_modules=True,
    finetune_mlp_modules=True,

    r=64,
    lora_alpha=64,
    lora_dropout=0.05,
    bias='none',
    random_state=3407,
    use_rslora=True,
    loftq_config=None
)

FastVisionModel.for_training(model)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=UnslothVisionDataCollator(model, tokenizer),
    train_dataset=training_messages,
    eval_dataset=validation_messages,
    args=SFTConfig(
        eval_strategy='steps',
        learning_rate=1e-4,
        weight_decay=0.01,
        lr_scheduler_type='constant',
        seed=3407,
        bf16=is_bf16_supported(),
        fp16=not is_bf16_supported(),
        remove_unused_columns=False,
        optim='paged_adamw_32bit',
        report_to='tensorboard',
        logging_steps=0.05,
        output_dir='outputs',
        dataset_text_field='',
        dataset_kwargs={'skip_prepare_dataset': True},
        dataset_num_proc=4,
        max_seq_length=2048
    ),
)

### Treinamento

In [ ]:
trainer_stats = trainer.train()

In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")

### Salvamento

In [ ]:
peft_model = trainer.model

model.save_pretrained(f'../weights/LLaMA_DERM_QLoRA_{training_dataset_size}_11B')
tokenizer.save_pretrained(f'../weights/LLaMA_DERM_QLoRA_{training_dataset_size}_11B')